In [2]:
import math
import tkinter as tk
from tkinter import *
from PIL import Image, ImageDraw

pixel_size = 20
canvas_resolution = [112, 73]
design_resolution = [300, 200]
canvas_width, canvas_height = [1120, 730]
corner_row, corner_column = [0, 0]
color_list = ["#0000FF", "#00FF00", "#FF0000", "#00FFFF", "#FFFF00", "#FF00FF", "#000000", "#FFFFFF", "#5486A5" ]
current_color = color_list[-1]
monitoring_buttons = []
cellDict = { 'id' : [], 'color' : [], 'row' : [], 'column' : [] }
tempStore = { 'id' : [], 'color' : [],  'row' : [], 'column' : [] }
gridList = []
horizontal_value = 0
vertical_value = 0
sliders = []

def adaptSlider():
    horizontal_scale = (canvas_width/pixel_size / design_resolution[0])      
    vertical_scale = (canvas_height/pixel_size / design_resolution[1]) 
    sliders[0].configure(sliderlength=horizontal_scale*1075)
    sliders[1].configure(sliderlength=vertical_scale*734)
#############  Slider to Wspace  START    ###################
def move_workspace():
    global corner_row
    global corner_column
    horizontal_slider, vertical_slider = sliders
    wSpaceWidth = canvas_width/pixel_size
    wSpaceHeight = canvas_height/pixel_size
    hSlideRange, wSlideRange = [100, 100]
    totalWidth ,totalHeight = design_resolution
    hTravel = totalWidth - wSpaceWidth
    vTravel = totalHeight - wSpaceHeight
    corner_row = math.floor(horizontal_slider.get()/hSlideRange*hTravel)
    corner_column = math.floor(vertical_slider.get()/vSlideRange*vTravel)
#############  Slider to Wspace  END  ###################
def create_sliders(screen_width, screen_height):
    horizontalLeftButton = tk.Button(root, height=1, width=1, text="<", command=move_left)
    horizontalLeftButton.place(x= screen_width*0.25-15, y = screen_height-100)
    horizontalSlider = tk.Scale(root, from_=0, to=100, orient="horizontal", length=1095, sliderlength= 40, command=horizontal_slider_change, showvalue=0)  
    horizontalSlider.place(x=screen_width*0.25, y=screen_height-100)
    horizontalRightButton = tk.Button(root, height=1, width=1, text=">", command=move_right)
    horizontalRightButton.place(x= screen_width*0.96, y = screen_height-100)
    verticalUpButton = tk.Button(root, height=1, width=1, text="U", command=move_up)
    verticalUpButton.place(x= screen_width-25, y = 10)
    verticalSlider = tk.Scale(root, from_=0, to=100, orient="vertical", length=734, sliderlength= 40, command=vertical_slider_change, showvalue=0)  
    verticalSlider.place(x=screen_width-25, y= 35)
    verticalDownButton = tk.Button(root, height=1, width=1, text="D", command=move_down)
    verticalDownButton.place(x= screen_width-25, y = screen_height-100)
    horizontalSlider.set(0)
    verticalSlider.set(0)
    global sliders
    sliders.append(horizontalSlider)
    sliders.append(verticalSlider)
    adaptSlider()
def horizontal_slider_change(value):
    global horizontal_value
    horizontal_value = value
    move_workspace()
    move_cells_in_workspace()
def vertical_slider_change(value):
    global vertical_value
    vertical_value = value
    move_workspace()
    move_cells_in_workspace()
def move_right():
    sliders[0].set(sliders[0].get()+5)
    global horizontal_value
    horizontal_value = sliders[0].get()
    move_workspace()
    move_cells_in_workspace()
def move_left():
    sliders[0].set(sliders[0].get()-5)
    global horizontal_value
    horizontal_value = sliders[0].get()
    move_workspace()
    move_cells_in_workspace()
def move_up():
    sliders[1].set(sliders[1].get()-5)
    global vertical_value
    vertical_value = sliders[1].get()
    move_workspace()
    move_cells_in_workspace()
def move_down():
    sliders[1].set(sliders[1].get()+5)
    global vertical_value
    vertical_value = sliders[1].get()
    move_workspace()
    move_cells_in_workspace()
def make_window_fit_screen(window):
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    window.geometry(f"{screen_width}x{screen_height}")
    return [screen_width, screen_height]
def clear_cell():
    for cell in cellDict['id']:
        canvas.delete(cell)
    global tempStore 
    tempStore = cellDict.copy()
    for key in cellDict:
        cellDict[key] = []
def restore_design():
    global cellDict 
    if len(cellDict['id']) != 0:
        return
    cellDict= tempStore.copy()
    for i in range(len(cellDict['id'])):
        new_id = canvas.create_rectangle(cellDict['row'][i]*pixel_size, cellDict['column'][i]* pixel_size,(cellDict['row'][i] + 1) * pixel_size, (cellDict['column'][i] + 1) * pixel_size, fill=cellDict['color'][i], outline="", tags='blocks')           
        cellDict['id'][i] = new_id
    for key in tempStore:
        tempStore[key] = []
def update_button(monitoringButtons):
    keys = list(cellDict.keys())
    for button in range(len(monitoringButtons)):
        monitoringButtons[button].config(text=cellDict[keys[button]][-1]) 
def on_canvas_click(event):
    x = event.x // pixel_size
    y = event.y // pixel_size
    cell_id = canvas.create_rectangle(x * pixel_size, y * pixel_size,(x + 1) * pixel_size, (y + 1) * pixel_size, fill=current_color, outline="", tags='blocks')      
    cellDict['id'].append(cell_id)
    cellDict['row'].append(x)
    cellDict['column'].append(y)
    cellDict['color'].append(current_color)
    update_button(monitoring_buttons)
def change_color(new_color):
    global current_color
    current_color = new_color 
    print(new_color)
def create_cell_monitor():
    keys= list(cellDict.keys())
    num_columns = 2
    xspacing = 100
    yspacing = 30
    start_x = 10
    start_y = 180
    for key in range(len(keys)):
        button = tk.Button(root, width=10, height=1, text=keys[key])
        monitoring_buttons.append(button)
        button.place(x=start_x + xspacing*(key%num_columns)  , y=start_y + yspacing*int(key/num_columns))
    return monitoring_buttons
def modify_workspace():
    clearButton = tk.Button(root, text="Clear All", command=clear_cell)
    clearButton.place(x=120, y = 130)
    restoreButton = tk.Button(root, text="Restore", command=restore_design)
    restoreButton.place(x=200, y = 130)
def zoom_in():
    zoom_in_button.configure(text="Zooming in")
    global pixel_size
    if pixel_size < 50:
        pixel_size += 5
    canvas.delete('gridlines')
    canvas.delete('blocks')
    create_gridlines(canvas, "#333333")
    for i in range(len(cellDict['id'])):
        new_id = canvas.create_rectangle(cellDict['row'][i]*pixel_size, cellDict['column'][i]* pixel_size,(cellDict['row'][i] + 1) * pixel_size, (cellDict['column'][i] + 1) * pixel_size, fill=cellDict['color'][i], outline="", tags='blocks')            
        cellDict['id'][i] = new_id
    zoom_in_button.configure(text="Zoom in")
    adaptSlider()
    move_workspace()
def zoom_out():
    zoom_out_button.configure(text="Zooming out")
    global pixel_size
    if pixel_size > 5:
        pixel_size -= 5
    canvas.delete('gridlines')
    canvas.delete('blocks')
    create_gridlines(canvas, "#333333")
    for i in range(len(cellDict['id'])):
        new_id = canvas.create_rectangle(cellDict['row'][i]*pixel_size, cellDict['column'][i]* pixel_size,(cellDict['row'][i] + 1) * pixel_size, (cellDict['column'][i] + 1) * pixel_size, fill=cellDict['color'][i], outline="", tags='blocks')          
        cellDict['id'][i] = new_id
    zoom_out_button.configure(text="Zoom out")
    adaptSlider()
    move_workspace()
def create_canvas(screen_width, screen_height, bgcolor, callback_for_mouse_click):
    global canvas
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg=bgcolor)
    canvas.place(x= 370, y = 10)
    canvas.bind("<Button-1>", callback_for_mouse_click)
    return canvas
def create_gridlines(canvas, gridcolor):
    if design_resolution[1]*pixel_size < canvas_height:
        for i in range(0, (design_resolution[1]+1)*pixel_size, pixel_size):
            gridList.append(canvas.create_line(0, i,  design_resolution[0]*pixel_size, i, fill=gridcolor, tags='gridlines'))
    else:
        for i in range(0, canvas_height, pixel_size):
            gridList.append(canvas.create_line(0, i,  canvas_width, i, fill=gridcolor, tags= 'gridlines'))   
    if design_resolution[0]*pixel_size < canvas_width:
        for i in range(0, (design_resolution[0]+1)*pixel_size, pixel_size):
            gridList.append(canvas.create_line(i, 0, i,  design_resolution[1]*pixel_size, fill=gridcolor, tags='gridlines'))
    else:
        for i in range(0, canvas_width, pixel_size):
            gridList.append(canvas.create_line(i, 0, i,  canvas_height, fill=gridcolor, tags='gridlines'))      

###########  move_cells_in_workspace start  ##############
def move_cells_in_workspace():
    cell_data = cellDict.copy()
    current_width, current_height = [canvas_width//pixel_size, canvas_height//pixel_size]
    for i in range(len(cell_data['row'])):
        cell_data['row'][i] -= corner_row
        cell_data['column'][i] -= corner_column
    global canvas
    canvas.delete('gridlines')
    canvas.delete('blocks')
    canvas.delete('tempBlocks')
    create_gridlines(canvas, "#333333")
    for i in range(len(cell_data['id'])):
        if cell_data['row'][i] >= 0 and cell_data['row'][i] < current_height and cell_data['column'][i] >= 0 and cell_data['column'][i] < current_width:   
            canvas.create_rectangle(cell_data['row'][i]*pixel_size, cell_data['column'][i]* pixel_size,(cell_data['row'][i] + 1) * pixel_size, (cell_data['column'][i] + 1) * pixel_size, fill=cell_data['color'][i], outline="", tags='tempBlocks')          
    cell_data.clear()
###########  move_cells_in_workspace end    ##############
            
            
root = tk.Tk()
root.configure(bg='#333333')
root.title("Pixel Art Canvas")




##### Color Swatches Start   ########
color_swatches = [
    tk.Button(root, height = 1, width = 2, bg = color_list[0], command= lambda: change_color(color_list[0])),   
    tk.Button(root, height = 1, width = 2, bg = color_list[1], command= lambda: change_color(color_list[1])),  
    tk.Button(root, height = 1, width = 2, bg = color_list[2], command= lambda: change_color(color_list[2])),  
    tk.Button(root, height = 1, width = 2, bg = color_list[3], command= lambda: change_color(color_list[3])),  
    tk.Button(root, height = 1, width = 2, bg = color_list[4], command= lambda: change_color(color_list[4])),  
    tk.Button(root, height = 1, width = 2, bg = color_list[5], command= lambda: change_color(color_list[5])),  
    tk.Button(root, height = 1, width = 2, bg = color_list[6], command= lambda: change_color(color_list[6])),  
    tk.Button(root, height = 1, width = 2, bg = color_list[7], command= lambda: change_color(color_list[7])),
    tk.Button(root, height = 1, width = 2, bg = color_list[8], command= lambda: change_color(color_list[8]))
]
for i in range(len(color_list)):
    color_swatches[i].place(x= 10 + 30*(i%3) , y= 250 + 30*int(i/3))
##### Color Swatches END   ######## 

######### Zoom Setting START  ############
zoom_in_button = tk.Button(root, text="Zoom in", width=15, height= 1, command=zoom_in)
zoom_in_button.place(x= 10, y= 350)
zoom_out_button = tk.Button(root, text="Zoom out", width=15, height= 1, command=zoom_out)
zoom_out_button.place(x= 150, y= 350)
########## Zoom Setting END   ############

###### Modify Workspace START  ##############
def get_resolution():
    global design_resolution
    design_resolution[0] = int(CanvasWidthEntry.get())
    design_resolution[1] = int(CanvasHeightEntry.get())
    canvas.delete('gridlines')
    create_gridlines(canvas, '#333333')
    adaptSlider()
CanvasWidthLabel = tk.Label(root, text="Canvas Width in Cells")
CanvasWidthLabel.place(x=10, y = 40)
CanvasWidthEntry = tk.Entry(root, width=15)
CanvasWidthEntry.place(x=150, y = 40)
CanvasHeightLabel = tk.Label(root, text="Canvas Height in Cells")
CanvasHeightLabel.place(x=10, y = 70)
CanvasHeightEntry = tk.Entry(root, width=15)
CanvasHeightEntry.place(x=150, y = 70)
SetResolutionButton = tk.Button(root, text="Set Resolution", command=get_resolution)
SetResolutionButton.place(x=120, y = 100)
###### Modify Workspace END  ##############

screen_width, screen_height = make_window_fit_screen(root)
canvas = create_canvas(screen_width, screen_height, 'white', on_canvas_click)
create_gridlines(canvas, '#333333')

buttons = create_cell_monitor()
create_sliders(screen_width, screen_height)
modify_workspace()

root.mainloop()

#0000FF


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_12444\3333430423.py", line 67, in vertical_slider_change
    move_workspace()
  File "C:\Users\User\AppData\Local\Temp\ipykernel_12444\3333430423.py", line 38, in move_workspace
    corner_column = math.floor(vertical_slider.get()/vSlideRange*vTravel)
                                                     ^^^^^^^^^^^
NameError: name 'vSlideRange' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_12444\3333430423.py", line 67, in vertical_slider_change
    move_workspace()
  File "C:\Users\User\AppData\Local\Temp\ipykernel_124

In [154]:
import tkinter as tk
from tkinter import *
import random
import math
import time
row = 0
column = 0
vSlide= random.randint(0, 100)
hSlide= random.randint(0, 100)
hSlideRange=100
vSlideRange=100
cellSize = 10
totalWidth, totalHeight = [350, 250]
wspaceWidth, wspaceHeight = [60, 40]
hTravel = totalWidth-wspaceWidth
vTravel = totalHeight-wspaceHeight
def scale_slider():
    global row
    global column
    row = math.floor(hSlide/hSlideRange*hTravel)
    column = math.floor(vSlide/vSlideRange*vTravel)
    print(row, column)
app = tk.Tk()
window = tk.Canvas(app, width=800, height=500, bg='white')
external_rectangle = window.create_rectangle(100, 100, totalWidth+100, totalHeight+100, fill= 'orange')
scale_slider()
internal_rectangle = window.create_rectangle(row+100, column+100, row+160, column+140, fill='blue')
window.pack()
app.mainloop()

194 79
